In [12]:
!pip install pycountry_convert

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681896 sha256=00a534a0df370692f18c663518e8f5f98716bfc0754b2c1dcb02f29e0f014c47
  Stored in directory: c:\users\wkosi\appdata\local\pip\cache\wheels\e2\aa\0f\c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import train test split fromo sklearn
from sklearn.model_selection import train_test_split
import pycountry_convert
#from pipeline_functions import *
import copy
import warnings

from pipeline_functions import *

In [2]:
df = pd.read_excel('online_retail_II.xlsx')

In [3]:
df = initial_transform(df)
df = add_continent_and_eu_columns(df)

df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Month,Year,Day,TotalPrice,YearMonth,Continent
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,12,2009,1,83.40,2009-12,EU_UK
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,12,2009,1,81.00,2009-12,EU_UK
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,12,2009,1,81.00,2009-12,EU_UK
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,12,2009,1,100.80,2009-12,EU_UK
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,12,2009,1,30.00,2009-12,EU_UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,12,2010,9,5.90,2010-12,EU_UK
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,12,2010,9,3.75,2010-12,EU_UK
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,12,2010,9,3.75,2010-12,EU_UK
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,12,2010,9,7.50,2010-12,EU_UK


## Time Series Split

In [4]:
from sklearn.model_selection import TimeSeriesSplit

In [5]:
tscv = TimeSeriesSplit()

In [11]:
enumerate(tscv.split(df))

## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Step 1: Data preparation
# Assume `quantity_sold` is a pandas DataFrame with columns "date" and "quantity"
train_data = df.loc[df["InvoiceDate"] < "2022-01-01"]
test_data = df.loc[df["InvoiceDate"] >= "2022-01-01"]

# Step 2: Model selection
d = 1  # Assume the time series needs to be differenced once to be stationary
p = 1  # Assume there is one autoregressive term
q = 1  # Assume there is one moving average term

# Step 3: Model fitting
model = ARIMA(train_data["quantity"], order=(p, d, q))
result = model.fit()

# Step 4: Model evaluation
forecast = result.forecast(len(test_data))
mae = np.mean(np.abs(forecast - test_data["quantity"]))
mse = np.mean((forecast - test_data["quantity"]) ** 2)
rmse = np.sqrt(mse)

# Step 5: Model improvement (if necessary)
# Try different orders of p, d, q or use a different model altogether